**GET DATA**

Summary of this notebook: ...

Definition of Done: ...

In [ ]:
# Import google drive and select a root directory
import os
from google.colab import drive
drive.mount("/content/gdrive")

!ls -lht /content/gdrive

# ROOT_DIR = "/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/nasa_impact/cloudstreet_smaller/"
# !ls -lht $
ROOT_DIR = "/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code"
os.chdir(ROOT_DIR)

In [ ]:
import os
import numpy as np
import pickle
from tqdm import tqdm, tqdm_notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors

import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
# Get the filenames, feature list (embeddings), and classification for the feature extraction done in part 2
# Class_ids are binary. Cloud street or not. 

filenames = pickle.load(open("Features/customcolorizer_merced_combined_filenames.pkl",'rb'))
feature_list = pickle.load(open("Features/customcolorizer_merced_combined_features.pkl", 'rb'))

In [ ]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

In [ ]:
def classname(str):
    return str.split('/')[0]

# Helper function to get the classname and filename
def classname_filename(str):
    return str.split('/')[-2] + '/' + str.split('/')[-1]


def calculate_accuracy(feature_list, num_nearest_neighbors):
    correct_predictions = 0
    incorrect_predictions = 0
    neighbors = NearestNeighbors(n_neighbors=num_nearest_neighbors,
                                 algorithm='brute',
                                 metric='euclidean').fit(feature_list)
    for i in tqdm_notebook(range(len(feature_list))):
        distances, indices = neighbors.kneighbors([feature_list[i]])
        for j in range(1, num_nearest_neighbors):
            if (classname(filenames[i]) == classname(
                    filenames[indices[0][j]])):
                correct_predictions += 1
            else:
                incorrect_predictions += 1
        
    # Note, this is actually precision @k
    print(
        "Accuracy is ",
        round(
            100.0 * correct_predictions /
            (1.0 * correct_predictions + incorrect_predictions), 2))

    
    return round(
            100.0 * correct_predictions /
            (1.0 * correct_predictions + incorrect_predictions), 2)

In [ ]:
# Select the number of neighbors to check for accuracy measurement
num_neighbor_check = [2,3,4,5,10,20,30,40,50]
# num_neighbor_check = [2]

presion_k = []
for num_neighbor in num_neighbor_check:
    print(num_neighbor)
    presion_k.append(calculate_accuracy(feature_list, num_neighbor))


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(num_neighbor_check, presion_k, c='r', marker='D')
plt.plot(num_neighbor_check, presion_k)
plt.xlabel('Number of Neighbors', fontweight='bold')
plt.ylabel('Accuracy (%)', fontweight='bold')
plt.title('Precision at K', fontweight='bold')